In [13]:
## ---------------------------
##
## cross-mapping analysis
##
## Summary of the per genome-species pair analysis
##
## Authors: Daria Romanovskaia
##
## Date Created: 2021-10-28
##
##
## ---------------------------

In [14]:
source(file.path(Sys.getenv("CODEBASE"),"DNAmeth500species/src/00.0_init.R"))

Joining, by = "species"



In [15]:
wd = file.path(analysis_dir, "validation", "01_crossMapping","01.5_analysis")
setwd(wd)

In [16]:
dir.create("summary")

Warning message in dir.create("summary"):
“'summary' already exists”


As a first step we want to only go deeper in the analysis of the best of the genomes, the species was mapped to.

In [17]:
cm_stats <- fread("../01.4_crossMapping_stats/cross_mapping_stats.tsv")
head(cm_stats)

total_passed.V1,total_failed.V3,secondary_passed.V1,secondary_failed.V3,mapped_passed.V1,mapped_failed.V3,perc_mapped_passed,perc_mapped_failed,species,mapped_genome,color_class
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<chr>,<chr>,<chr>
1677300,0,50162,0,441133,0,26.30,NA,ABB,canFam4,Mammalia
1677300,0,44708,0,482219,0,28.75,NA,ABB,musFur1,Mammalia
1677300,0,46859,0,701722,0,41.84,NA,ABB,neoSch1,Mammalia
1924470,0,35377,0,156209,0,8.12,NA,AB,anoCar2,Reptilia
1924470,0,24986,0,110258,0,5.73,NA,AB,thaSir1,Reptilia
1820594,0,92172,0,365757,0,20.09,NA,ABU,nanPar1,Amphibia


In [18]:
cross_mapping_stats_best <- cm_stats %>% arrange(desc(perc_mapped_passed)) %>%
group_by(species) %>%
filter(perc_mapped_passed == first(perc_mapped_passed))
setDT(cross_mapping_stats_best)

In [19]:
head(cross_mapping_stats_best)

total_passed.V1,total_failed.V3,secondary_passed.V1,secondary_failed.V3,mapped_passed.V1,mapped_failed.V3,perc_mapped_passed,perc_mapped_failed,species,mapped_genome,color_class
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<chr>,<chr>,<chr>
1318577,0,51861,0,1294735,0,98.19,NA,CM,calJac4,Mammalia
2387998,0,55236,0,2341500,0,98.05,NA,NM,hetGla2,Mammalia
2301812,0,69263,0,2252356,0,97.85,NA,WB,susScr11,Mammalia
1540769,0,43629,0,1502856,0,97.54,NA,RAT,rn7,Mammalia
986291,0,30695,0,960181,0,97.35,NA,FR,rn7,Mammalia
1361285,0,38209,0,1323429,0,97.22,NA,DOG,canFam5,Mammalia


In [20]:
cross_mapping_stats_best[, folder:=paste0(species, "_", mapped_genome), by = row.names(cross_mapping_stats_best)]

In [9]:
summary_stats = data.table()
for(i in 1:NROW(cross_mapping_stats_best)){
    
    df <- try(fread(file.path(cross_mapping_stats_best$species[[i]], cross_mapping_stats_best$folder[[i]], 
                          paste0(cross_mapping_stats_best$species[[i]], "_methylation_profile_stats.tsv"))))
    if(is.list(df)){
    df$mapped_genome <- cross_mapping_stats_best$mapped_genome[[i]]
    df$color_class <- cross_mapping_stats_best$color_class[[i]]
        
    summary_stats <- rbind(summary_stats, df)
        
    }else{
        print(cross_mapping_stats_best$folder[[i]])
    }
    
}

Error in fread(file.path(cross_mapping_stats_best$species[[i]], cross_mapping_stats_best$folder[[i]],  : 
  File 'BSN/BSN_bisBis1/BSN_methylation_profile_stats.tsv' does not exist or is non-readable. getwd()=='/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis'
[1] "BSN_bisBis1"
Error in fread(file.path(cross_mapping_stats_best$species[[i]], cross_mapping_stats_best$folder[[i]],  : 
  File 'OS/OS_aptMan1/OS_methylation_profile_stats.tsv' does not exist or is non-readable. getwd()=='/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis'
[1] "OS_aptMan1"
Error in fread(file.path(cross_mapping_stats_best$species[[i]], cross_mapping_stats_best$folder[[i]],  : 
  File 'RH/RH_aptMan1/RH_methylation_profile_stats.tsv' does not exist or is non-readable. getwd()=='/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis'
[1] "RH_aptMan1"
Error in fread(file.path(cr

In [18]:
head(summary_stats)

label,y95_Downstream,y95_Transcript,y95_Upstream,ymin_Upstream,xmin_Upstream,mapped_genome,color_class,species
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CM_1_H,67.66037,76.99387,62.68334,15.023332,-4.037975,calJac4,Mammalia,CM
CM_1_L,67.00738,76.82105,62.97616,13.551712,-4.037975,calJac4,Mammalia,CM
CM_2_H,74.04171,79.58644,72.54984,28.024164,-4.037975,calJac4,Mammalia,CM
CM_2_L,81.41495,86.27358,81.88591,46.122194,-4.037975,calJac4,Mammalia,CM
NM_1_H,58.85295,70.48016,57.17615,8.447119,-4.037975,hetGla2,Mammalia,NM
NM_1_L,63.46335,74.42746,60.43386,11.219202,-4.037975,hetGla2,Mammalia,NM


In [13]:
summary_stats <- summary_stats[label %in% stats_annot$Sample_Name,]

In [17]:
summary_stats[, species:=strsplit(label, "_")[[1]][1], by = row.names(summary_stats)]

In [16]:
head(cross_mapping_stats_best)

total_passed.V1,total_failed.V3,secondary_passed.V1,secondary_failed.V3,mapped_passed.V1,mapped_failed.V3,perc_mapped_passed,perc_mapped_failed,species,mapped_genome,color_class,folder
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>
1318577,0,51861,0,1294735,0,98.19,NA,CM,calJac4,Mammalia,CM_calJac4
2387998,0,55236,0,2341500,0,98.05,NA,NM,hetGla2,Mammalia,NM_hetGla2
2301812,0,69263,0,2252356,0,97.85,NA,WB,susScr11,Mammalia,WB_susScr11
1540769,0,43629,0,1502856,0,97.54,NA,RAT,rn7,Mammalia,RAT_rn7
986291,0,30695,0,960181,0,97.35,NA,FR,rn7,Mammalia,FR_rn7
1361285,0,38209,0,1323429,0,97.22,NA,DOG,canFam5,Mammalia,DOG_canFam5


In [20]:
summary_stats <- left_join(summary_stats, cross_mapping_stats_best[, c("mapped_genome", "species", "perc_mapped_passed")])

Joining, by = c("mapped_genome", "species")



In [21]:
my_wt(summary_stats, "summary/Methylation_profile_stats_summary.tsv")

In [22]:
getwd()

[1] "/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis"

In [ ]:
"/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis/CM/CM_calJac4/"

In [21]:
full_stats = data.table()
for(i in 1:NROW(cross_mapping_stats_best)){
    
    df <- try(fread(file.path(cross_mapping_stats_best$species[[i]], cross_mapping_stats_best$folder[[i]], 
                          paste0(cross_mapping_stats_best$species[[i]], "_methylation_profile.tsv"))))
    if(is.list(df)){
    df$mapped_genome <- cross_mapping_stats_best$mapped_genome[[i]]
    df$color_class <- cross_mapping_stats_best$color_class[[i]]
        
    full_stats <- rbind(full_stats, df)
        
    }else{
        print(cross_mapping_stats_best$folder[[i]])
    }
    
}

In [22]:
object.size(full_stats)

9994393096 bytes

In [23]:
NROW(full_stats)

[1] 155205634

In [24]:
my_wt(full_stats, "/binfl/lv71484/droman/DNAmeth500species/results_analysis/validation/01_crossMapping/01.5_analysis/summary/Methylation_profiles_merged.tsv")